In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import transbigdata as tbd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#获取行政区划
admin,district = tbd.getadmin(
    '广东省', 
    ak='169521e66b7a30adcaa36ae0a9c04c8d',
    jscode = '2cdd826ad4099fae293dbabf3657c78d',subdistricts = True)
admin.plot()

In [ ]:
import urllib
from urllib import request
import json

def get_migration(move_type = 'move_in',date = '20230116',city = '440500'):

    url = 'https://huiyan.baidu.com/migration/cityrank.jsonp?dt=city&id='+city+'&type='+move_type+'&date='+date
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    webpage = response.read()
    result = json.loads(webpage.decode('utf8','ignore')[4:-1])
    return pd.DataFrame(result['data']['list'])

def get_index(move_type = 'move_in',date = '20230116',city = '440500'):
    url = 'http://huiyan.baidu.com/migration/historycurve.jsonp?dt=city&id='+city+'&type='+move_type
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    webpage = response.read()
    result = json.loads(webpage.decode('utf8','ignore')[4:-1])
    return result['data']['list'][date]

def get_data(date):

    guangdong_district = district[['name','adcode']]
    def cal_od(r):
        cityname = r['name']
        citycode = r['adcode']
        print('get ',cityname,date)
        move = get_migration('move_in',date,citycode)
        move.columns = ['O','O_province','prop']
        move['D'] = cityname
        move['index'] = get_index('move_in',date,citycode)
        move['value'] = move['prop']*move['index']/100
        return move
    result = guangdong_district.apply(lambda r:cal_od(r),axis=1)
    result = pd.concat(list(result))
    result['date'] = date

    district['lon'] = district['center'].apply(lambda x:float(x.split(',')[0]))
    district['lat'] = district['center'].apply(lambda x:float(x.split(',')[1]))
    o = district[['name','lon','lat']]
    o.columns = ['O','slon','slat']
    d = district[['name','lon','lat']]
    d.columns = ['D','elon','elat']
    od = pd.merge(pd.merge(result,o),d)
    print('get data success')
    return od

In [ ]:
#dates =[(pd.to_datetime('20230125')+pd.Timedelta(days=i)).strftime('%Y%m%d') for i in range(30)]
dates = ['20230125', '20230126', '20230127', '20230128', '20230129', '20230130', '20230131', '20230201', '20230202', '20230203', '20230204', '20230205', '20230206', '20230207', '20230208', '20230209', '20230210', '20230211', '20230212', '20230213', '20230214', '20230215', '20230216']
for date in dates:
    # 存储数据
    od = get_data(date)
    od.to_csv('od/od'+date+'.csv',index=False)